In [1]:
#do this later from admin: !python -m pip install --upgrade pip


# TOC

0. [Imports](#first-bullet)
1. [EDA](#1.0-bullet)
2. [Feature Engineering](#2.0-bullet)  
 2.1 [Pt 1 - Regression](#2.1-bullet)  
 2.2 [Pt 2 - Classification](#2.2-bullet)
3. [Basic Model](#3.0-bullet)  
 3.1 [Pt 1 - Regression](#3.1-bullet)  
 3.2 [Pt 2 - Classification](#3.2-bullet)
4. [Reference and Planning](#4.0-bullet)

In [10]:
#Basic
import pandas as pd
import numpy as np
#Math
import scipy as sp
from random import randint
from math import exp
import operator

#Scraping
import requests
import json

#Youtube Data API Packages: 
"""from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
from googleapiclient.discovery import build
import argparse
from googleapiclient.errors import HttpError"""

#Modeling
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import r2_score,confusion_matrix, roc_auc_score, roc_curve

#NLP & Text Management
# from sklearn.feature_extraction import stop_words
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
# import re
# from textblob import TextBlob
# from wordcloud import WordCloud
# from nltk.stem import PorterStemmer

#Time analysis
import time
import datetime
# from statsmodels.tsa.stattools import pacf
# from statsmodels.graphics.tsaplots import plot_pacf

#Plotting
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud

%matplotlib inline
plt.style.use('classic')
sns.set_style('whitegrid')

In [78]:
df_main = pd.read_csv('./data/engineered_data/data_genres.csv')

In [79]:
model_time = ['is_weekend', 'is_friday', 'year_day', 'week', 'week_day', 'month', 'month_day', 'year',  ]
model_text = ['description_sentiment', 'description_wordcount',
  'tags_sentiment', 'tags_wordcount', 'title_sentiment',
 'title_wordcount', 'title_featuring', 'title_length', ]
model_title_letter = ['intitle_a',  'intitle_b', 'intitle_c', 'intitle_d', 'intitle_e', 'intitle_f', 'intitle_g',
 'intitle_h', 'intitle_i', 'intitle_j', 'intitle_k', 'intitle_l', 'intitle_m', 'intitle_n', 'intitle_o',
 'intitle_p', 'intitle_q', 'intitle_r', 'intitle_s', 'intitle_t', 'intitle_u', 'intitle_v', 'intitle_w',
 'intitle_x', 'intitle_y', 'intitle_z',]              
model_ratios = ['lv_ratio', 'comment_view_ratio', 'like_dislike_ratio',]
model_other = ['duration', 'contentRating', 'regionRestriction',  'licensedContent', 'caption', 'definition']
measures = ['likeCount', 'dislikeCount', 'commentCount', 'viewCount',
            'view_log',  'comment_log',  'like_log', 'dislike_log', 
            'view_class' ]
non_model_cols = ['request_token', 'letter_search','channelTitle', 'tags', 'title', 'vidId','publishedAt','favoriteCount',
              'description','defaultLanguage', ]

In [80]:
genre_list_str = ['alt_words', 'blues_words', 'christian_words', 'classical_words', 'country_words', 'dubstep_words', 
                 'edm_words', 'extended_words', 'folk_words', 'halloween_words', 'hit_words', 'italian_words', 'jazz_words', 
                 'kpop_words', 'latin_words', 'love_songs', 'other_rock_words', 'pop_words', 'rap_words', 'remix_words', 
                 'reggae_words', 'romanian_words', 'relax_words', 'rock_words', 'trap_words']
deltas_list = ['like_baseline_delta', 'comment_baseline_delta', 'dislike_baseline_delta', 'ld_baseline_delta' ]

In [81]:
baseline_stats = pd.read_csv('./data/engineered_data/engagement_matrix.csv')

In [82]:
df_formodel = df_main[(df_main.view_log>4)&(df_main.lv_ratio>0)&(df_main.comment_view_ratio>0)&(df_main.like_dislike_ratio<10)]

In [83]:
df_formodel.columns

Index(['index', 'caption', 'channelTitle', 'commentCount', 'defaultLanguage',
       'definition', 'description', 'dislikeCount', 'duration',
       'favoriteCount', 'licensedContent', 'likeCount', 'publishedAt',
       'regionRestriction', 'tags', 'title', 'vidId', 'viewCount',
       'request_token', 'letter_search', 'contentRating', 'year', 'month',
       'month_day', 'year_day', 'week_day', 'view_log', 'comment_log',
       'like_log', 'dislike_log', 'view_class', 'week',
       'description_sentiment', 'description_wordcount', 'tags_sentiment',
       'tags_wordcount', 'title_sentiment', 'title_wordcount',
       'title_featuring', 'title_length', 'lv_ratio', 'comment_view_ratio',
       'like_dislike_ratio', 'is_weekend', 'is_friday', 'alt_words',
       'blues_words', 'christian_words', 'classical_words', 'country_words',
       'dubstep_words', 'edm_words', 'extended_words', 'folk_words',
       'halloween_words', 'hit_words', 'italian_words', 'jazz_words',
       'kpop_words'

# <a class="anchor" id="1.0-bullet">1. Basic Grouping</a>

Do this first, then go back and do it without converting to logs, if meaningful

In [84]:
#Engagement per second of content? See diminishing returns

In [85]:
like_dict ={}
comment_dict = {}
dislike_dict = {}
ld_dict = {}
view_dict = {}
duration_dict = {}
for genre in genre_list_str:
    like_dict[genre.replace('_words', '')] = df_formodel[df_formodel[genre]==1].like_baseline_delta.mean()
    comment_dict[genre.replace('_words', '')] = df_formodel[df_formodel[genre]==1].comment_baseline_delta.mean()
    dislike_dict[genre.replace('_words', '')] = df_formodel[df_formodel[genre]==1].dislike_baseline_delta.mean()
    ld_dict[genre.replace('_words', '')] = df_formodel[df_formodel[genre]==1].ld_baseline_delta.mean()
    view_dict[genre.replace('_words', '')] = df_formodel[df_formodel[genre]==1].view_log.mean()
    duration_dict[genre.replace('_words', '')] = df_formodel[df_formodel[genre]==1].duration.mean()

In [86]:
df_engagement = pd.concat([
    pd.Series(like_dict), pd.Series(comment_dict), pd.Series(dislike_dict), pd.Series(ld_dict), pd.Series(view_dict), pd.Series(duration_dict)]
    , axis=1)

In [87]:
df_engagement.columns=['likes', 'comments', 'dislikes', 'likes_dislikes', 'views', 'duration']

In [88]:
df_engagement.sort_values(by='likes', ascending=False)

,likes,comments,dislikes,likes_dislikes,views,duration
jazz,0.678896,1.158578,-0.657110,8.981153,4.298613,64.000000
classical,0.643343,0.286811,0.743747,-1.418765,5.754542,1333.038462
reggae,0.490448,0.684599,0.651843,-17.344890,6.605652,252.666667
other_rock,0.395931,-0.169171,-0.277656,4.475771,6.242437,332.500000
country,0.378359,0.534521,0.146548,-1.105841,6.806138,328.500000
romanian,0.305963,0.467147,-0.003400,2.144092,6.806652,230.735294
italian,0.265597,0.529893,-0.026770,2.462043,6.744561,318.437500
blues,0.229378,0.106023,0.238421,0.111303,7.212179,316.933333
hit,0.126122,0.076246,0.254928,-1.232357,7.662304,259.024444
rock,0.117103,0.029970,0.220484,-0.333633,6.824143,429.356643


In [93]:
df_main[df_main.title.str.contains('White Noise')].kpop_words

1873    False
Name: kpop_words, dtype: bool

# <a class="anchor" id="1.0-bullet">1. Functions</a>

In [94]:
df_forclass = df_formodel.loc[:, genre_list_str + deltas_list] #

In [96]:
from sklearn.cluster import DBSCAN, KMeans

In [97]:
from sklearn.metrics import silhouette_score

In [98]:
ss = StandardScaler()
df_forclass_sc = ss.fit_transform(df_forclass)

In [105]:
len(genre_list_str)

25

In [ ]:
n_dict = {}
for n in range(3, 40): 
    k_classifier = KMeans(n_clusters=n)
    n_dict[n] = silhouette_score(df_forclass_sc, k_classifier.fit_predict(df_forclass_sc)) 
    print('   n: ', n, ' silhouette: ', n_dict[n], ' intertia: ' , k_classifier.inertia_)

   n:  3  silhouette:  0.14757297038525594  intertia:  193058.12390977042
   n:  4  silhouette:  0.4398561955047074  intertia:  187390.25440602534
   n:  5  silhouette:  0.16828426506155864  intertia:  178169.89797220659
   n:  6  silhouette:  0.17205884371768024  intertia:  170671.71935683695
   n:  7  silhouette:  0.1562288565016387  intertia:  164016.30467929717
   n:  8  silhouette:  0.18219527736513708  intertia:  156481.56437521157
   n:  9  silhouette:  0.19459814504422585  intertia:  148547.75608775104
   n:  10  silhouette:  0.20479493276623134  intertia:  141091.46883729633
   n:  11  silhouette:  0.18872014146191163  intertia:  134102.39511918768
   n:  12  silhouette:  0.2155236387198299  intertia:  128374.26148602128
   n:  13  silhouette:  0.19530043455497909  intertia:  119735.07322087736
   n:  14  silhouette:  0.1928832281796545  intertia:  111681.27185525533
   n:  15  silhouette:  0.22207623112250946  intertia:  105463.71112057948
   n:  16  silhouette:  0.2246474935

0.34200194468205614

In [103]:
#['tags_sentiment', 'description_sentiment', 'description_wordcount', 'title_featuring', 'duration', 'year', 'title_length', 'lv_ratio', 'view_log']

### DBScan

In [18]:
db_model = DBSCAN()

pd.DataFrame(db_model.fit_predict(uns_X_sc), columns=['class'])['class'].value_counts()

-1     8529
 0       15
 7        7
 5        7
 3        7
 10       7
 2        7
 1        6
 6        5
 4        5
 9        5
 8        2
Name: class, dtype: int64

In [138]:
db_preds = db_model.fit_predict(uns_X_sc)
df_db_classes = df_main[df_main.view_class>1]

In [139]:
df_db_classes['db_class'] = db_preds

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
